PARTE 3 - DETECTOR DE OBJETOS

Se deberá conseguir una red neuronal convolucional de clasificación pre-entrenada que sea capaz de distinguir entre un buen número de objetos diferentes en imágenes (ej: Las 1000 clases diferentes de imagenet). Se creará un modelo a partir de esta red, cuya inferencia reciba un directorio con los fotogramas de un vídeo en orden y la salida sea un dataframe de pandas con 3 columnas: el número de fotograma, la clase más probable detectada y el grado de confianza que da la red para dicha clasificación.

In [ ]:
import torch 
import torchvision.models as models # Para acceder a los modelos pre-entrenados
import torchvision.transforms as transforms # Para aplicar transformaciones a las imágenes
from PIL import Image # Para manipular imágenes
import pandas as pd # Para manipular datos
import os # Para crear carpetas y manejar rutas de archivos
import glob # Para buscar archivos
import logging # Para el manejo de logs

# Configuracion del logging: crear un archivo 'deteccion_objetos.log' para guardar los errores
logging.basicConfig(filename='deteccion_objetos.log', level=logging.INFO)
format='%(asctime)s %(levelname)s %(message)s'

# 1. Cargar el modelo (ResNet50) y ponerlo en modo evaluacion
try:
    modelo = models.resnet50(pretrained=True) # Cargar el modelo
    modelo.eval() # Ponerlo en modo evaluacion
except Exception as e: # Manejo de errores
    logging.error(f"Error al cargar el modelo: {e}") # Registrar el error
    exit() # Cierre del programa si no se ha podido cargar el modelo

# 2. Transformaciones para pre-procesar las imágenes
transformaciones = transforms.Compose([
    transforms.Resize(256), # Redimensionar las imágenes a 256x256
    transforms.CenterCrop(224), # Cortar las imágenes a 224x224
    transforms.ToTensor(), # Convertir a tensores
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalizar los valores
])



In [ ]:
def procesar_fotograma(ruta_imagen):
    """ Procesa un fotograma, devuelve la clase predicha y la confianza. """
    try:
        imagen = Image.open(ruta_imagen).convert("RGB") # Convertir a RGB
        imagen_tensor = transformaciones(imagen).unsqueeze(0) # Aplica las transformaciones y añade una dimensión de lote (batch)

        with torch.no_grad(): # Desactiva el calculo de gradientes
            salida = modelo(imagen_tensor) # Pasa la imagen por el modelo

        probabilidades = torch.nn.functional.softmax(salida, dim=1)[0] # Obtiene las probabilidades de la clase
        confianza, indice_prediccion = torch.max(probabilidades, dim=0) # Obtiene la clase y la confianza con mayor probabilidad

        return indice_prediccion.item(), confianza.item() # Devuelve la clase y la confianza
    except FileNotFoundError: # Manejo de errores si no se encuentra la imagen
        logging.error(f"No se encuentra la imagen {ruta_imagen}") # Registra el error
        return -1, 0.0 # Devuelve un valor por defecto
    except Image.UnidentifiedImageError: # Manejo de errores si no es una imagen
        logging.error(f"No es una imagen {ruta_imagen}") # Registra el error
        return -1, 0.0 # Devuelve un valor por defecto
    except Exception as e: # Manejo de errores generales
        logging.exception(f"Error al procesar la imagen {ruta_imagen}: {e}") # Registra el error
        return -1, 0.0 # Devuelve un valor por defecto


In [ ]:
def procesar_directorio_frames(directorio_frames):
    """ Procesa un directorio de fotogramas y devuelve un DataFrame
    """
    resultados = [] # Lista para almacenar los resultados
    archivos_frame = glob.glob(os.path.join(directorio_frames, "*.jpg")) # Usa glob para obtener solo los archivos jpg
    for archivo_frame in sorted(archivos_frame, key=lambda x: int(os.path.splitext(os.path.basename(x))[0])): # ordena con glob
        try:
            nombre_archivo = os.path.basename(archivo_frame) # Obtener el nombre del archivo
            indice, confianza = procesar_fotograma(archivo_frame) # Procesar el fotograma
            resultados.append([int(nombre_archivo[:-4]), indice, confianza]) # Agregar el resultado a la lista
        except Exception as e: # Manejo de errores
            logging.exception(f"Error al procesar el directorio {directorio_frames}: {e}")
    df_resultados = pd.DataFrame(resultados, columns=["Frame", "indice", "confianza"]) # Crear un DataFrame de pandas con los resultados
    return df_resultados # Devolver el DataFrame

# Rutas princiaples
ruta_frames = "frames" # Ruta a la carpeta que contiene las carpetas de las categorías
categorias = ["coches", "mascotas", "comida"] # Lista de las categorías

# Procesamiento principal
for categoria in categorias: # Iterar sobre las categorías
    ruta_categoria_frames = os.path.join(ruta_frames, categoria) # Ruta a la carpeta de frames de la categoría
    for video_carpeta in os.listdir(ruta_categoria_frames): # Iterar sobre las carpetas de frames de la categoría
        ruta_video_frames = os.path.join(ruta_categoria_frames, video_carpeta) # Ruta a la carpeta de frames de la categoría
        if os.path.isdir(ruta_video_frames): # Comprobar si es una carpeta
            try:
                df_resultados = procesar_directorio_frames(ruta_video_frames) # Procesar el directorio de frames
                df_resultados.to_csv(os.path.join(ruta_video_frames, "resultados.csv"), index=False) # Guardar el DataFrame en el archivo resultados.csv
                print(f"Resultados del video: {video_carpeta} de la categoría {categoria} guardados") # Imprimir mensaje de salida
            except Exception as e: # Manejo de errores
                logging.exception(f"Error al procesar la categoría {categoria} y video: {video_carpeta}: {e}") # Registrar el error